In [1]:
import importlib
from component import FeatureSelection
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [2]:
context = InteractiveContext()

In [3]:
import urllib.request
import tempfile
import os

# getting data and setup CsvExampleGen
DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-datagnrf_ugy/data.csv',
 <http.client.HTTPMessage at 0x7fa6ccc76990>)

In [4]:
example_gen = CsvExampleGen(input_base=DATA_ROOT)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-interactive-2021-09-17T21_52_43.474315-dirj7v5y/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:25648,xor_checksum:1631895763,sum_checksum:1631895763"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [5]:
# give path to the module file
feature_selector = FeatureSelection(orig_examples = example_gen.outputs['examples'],
                                   module_file="module_file")

In [6]:
context.run(feature_selector)

2021-09-17 21:52:45.313214: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-17 21:52:45.333196: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


ExecutionResult(
    component_id: FeatureSelection
    execution_id: 2
    outputs:
        feature_selection: Channel(
            type_name: Feature Selection
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-interactive-2021-09-17T21_52_43.474315-dirj7v5y/FeatureSelection/feature_selection/2"
        properties {
          key: "p_values"
          value {
            struct_value {
              fields {
                key: "culmen_depth_mm"
                value {
                  number_value: 0.1395146468107023
                }
              }
              fields {
                key: "culmen_length_mm"
                value {
                  number_value: 0.1452141956190967
                }
              }
              fields {
                key: "flipper_length_mm"
                value {
                  number_value: 0.1603053323396063
                }
              }
              fields {
                key: "species"
                value {
                  number_value: 0.07309942813170722
                }
              }
            }
          }
        }
        properties {
          key: "scores"
          value {
            struct_value {
              fields {
                key: "culmen_depth_mm"
                value {
                  number_value: 3.9391713763470286
                }
              }
              fields {
                key: "culmen_length_mm"
                value {
                  number_value: 3.8590908307969642
                }
              }
              fields {
                key: "flipper_length_mm"
                value {
                  number_value: 3.661349910337517
                }
              }
              fields {
                key: "species"
                value {
                  number_value: 5.23186947070533
                }
              }
            }
          }
        }
        properties {
          key: "selected_features"
          value {
            struct_value {
              fields {
                key: "__value__"
                value {
                  list_value {
                    values {
                      string_value: "species"
                    }
                    values {
                      string_value: "culmen_depth_mm"
                    }
                  }
                }
              }
            }
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "feature_selection"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "FeatureSelection"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "Feature Selection"
        properties {
          key: "p_values"
          value: STRUCT
        }
        properties {
          key: "scores"
          value: STRUCT
        }
        properties {
          key: "selected_features"
          value: STRUCT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_data: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 3
        type_id: 5
        uri: "/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-interactive-2021-09-17T21_52_43.474315-dirj7v5y/FeatureSelection/updated_data/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "updat

In [7]:
example_gen.outputs["examples"].get()[0].split_names

'["train", "eval"]'